# 유사도를 이용한 추천 시스템

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
data = pd.read_csv('./movie/movies_metadata.csv', low_memory=False)
data = data[:20000] # 데이터 수 줄이기

In [ ]:
'''
    코사인 유사도에 사용할 데이터: title, overview 칼럼
'''

In [4]:
# NULL 확인
data['overview'].isnull().sum()

135

In [5]:
data['overview'] = data['overview'].fillna('')

In [9]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
tfidf_matrix.shape # 20,000개의 영화를 표현하기 위해 총 47,487개의 단어가 사용됨.

(20000, 47487)

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [12]:
indices = pd.Series(data.index,  index=data['title']).drop_duplicates()
indices.head(5) # 영화 타이틀 - 인덱스 테이블 생성

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [22]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title] # 영화 타이틀에 해당하는 인덱스
    sim_scores = list(enumerate(cosine_sim[idx])) # 코사인 유사도
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # 정렬
    sim_scores = sim_scores[1:11] # 10개만 받아오기. 0번은 자기 자신
    movie_indices = [i[0] for i in sim_scores] # 10개의 인덱스
    return data['title'].iloc[movie_indices]

get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object